In [ ]:
import sklearn as skl
import sys
import numpy as np
import csv
import re
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

your_list= []
file1_1 = open("reviews.csv","r",encoding="utf8")
for line1_1 in file1_1: 
    your_list.append(line1_1[:-1])

file1_1.close()
your_list = your_list[1:]
your_list = your_list[0:1000]
your_label   = []
your_comment = []
 
for i in your_list:
    your_label.append(i.split('|')[0])
    your_comment.append(i.split('|')[1:])

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text
    
your_comment_1 = your_comment
   
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs  = np.array(your_comment_1)
bag   = count.fit_transform(docs.ravel())

from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

your_sentiment = []

for i in your_label:
    if i=='positive': your_sentiment.append(1)
    else: your_sentiment.append(0)

x_train = np.asarray(your_comment_1[0:800])
y_train = np.asarray(your_sentiment[0:800])
x_test  = np.asarray(your_comment_1[800:])
y_test  = np.asarray(your_sentiment[800:])

from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer(text):
    text=str(text)
    return text.split()
def tokenizer_porter(text):
    text=str(text)
    return [porter.stem(word) for word in text.split()]

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve



tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter]},
#               'clf__max_depth': [1, 5, 10]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None]}
#               'clf__max_depth': [1, 5, 10]},
              ]
dt_tfidf = Pipeline([('vect', tfidf),
                     ('clf',  AdaBoostClassifier(DecisionTreeClassifier(max_depth=2),
                                                 n_estimators=600,
                                                 learning_rate=1.5,
                                                 algorithm="SAMME"))])

gs_dt_tfidf = GridSearchCV(dt_tfidf, 
                           param_grid, 
                           cv=5,
                           verbose=1)


train_sizes, train_scores, test_scores =\
                learning_curve(estimator=gs_dt_tfidf,
                               X=x_train,
                               y=y_train,
                               train_sizes=np.linspace(0.1, 1.0, 10),
                               cv=10,
                               n_jobs=1)



train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_mean,
         color='blue', marker='o',
         markersize=5, label='training accuracy')

plt.fill_between(train_sizes,
                 train_mean + train_std,
                 train_mean - train_std,
                 alpha=0.15, color='blue')

plt.plot(train_sizes, test_mean,
         color='green', linestyle='--',
         marker='s', markersize=5,
         label='validation accuracy')

plt.fill_between(train_sizes,
                 test_mean + test_std,
                 test_mean - test_std,
                 alpha=0.15, color='green')

plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.8, 1.0])
plt.tight_layout()
# plt.savefig('./figures/learning_curve.png', dpi=300)
plt.show()



In [ ]:
#gs_dt_tfidf.fit(x_train,y_train)

In [ ]:
print('Best parameter set: %s ' % gs_dt_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_dt_tfidf.best_score_)

In [ ]:
clf  =  gs_dt_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(x_test, y_test))

In [ ]:
from sklearn.metrics import confusion_matrix

gs_dt_tfidf.fit(x_train, y_train)
y_pred = gs_dt_tfidf.predict(X_test)
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

from sklearn.metrics import precision_score, recall_score, f1_score

print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred))